# 0. Prework

In [ ]:
# 그래프에서 한글이 깨지지 않게 폰트 설치. 
# 맨처음 실행 후 런타임 다시시작해야 반영됨
# colab이라면 cell에서, Linux 등의 환경이라면 터미널 통해서 아래 코드 실행
'''
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
'''

'''
!pip install transformers
!pip install sentencepiece # MarianTokenizer 불러올 때 필요
!pip install sacremoses # MarianMTModel 에서 불러올 때 warning 뜨는 것 방지
'''

# 1. Load Data & Preview

In [ ]:
import pandas as pd
import torch

In [ ]:
data = pd.read_excel('./대화체.xlsx')
data.head()

In [ ]:
BATCH_SIZE = 128 ## 논문에선 2.5만 token이 한 batch에 담기게 했다고 함.
EPOCH = 20 ## 논문에선 약 560 에포크 진행
max_len = 512
d_model = 512

warmup_steps = 1500 ## 논문에선 4,000 스탭 
LR_scale = 1 # Noam scheduler에 peak LR 값 조절을 위해 곱해질 녀석 

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        return self.data.loc[idx, '원문'], self.data.loc[idx, '번역문']

data = pd.read_excel('대화체.xlsx')
custom_DS = CustomDataset(data)

train_DS, val_DS, test_DS, _ = torch.utils.data.random_split(custom_DS, [95000, 2000, 1000, len(custom_DS)-95000-2000-1000])

train_DL = torch.utils.data.DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
val_DL = torch.utils.data.DataLoader(val_DS, batch_size=BATCH_SIZE, shuffle=True)
test_DL = torch.utils.data.DataLoader(test_DS, batch_size=BATCH_SIZE, shuffle=True)

print(len(train_DS))
print(len(val_DS))
print(len(test_DS))

# 1. Import Libraries

In [ ]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
from tqdm import tqdm
import math
import matplotlib.pyplot as plt

import plotly.graph_objs as go

# 2.Load Tokenizer

In [ ]:
# Load tokenizer
tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ko-en')

eos_idx = tokenizer.eos_token_id
pad_idx = tokenizer.pad_token_id
print("eos_idx = ", eos_idx)
print("pad_idx = ", pad_idx)

In [ ]:
vocab_size = tokenizer.vocab_size
print(f'tokenizer의 사전 크기: {vocab_size}')

In [ ]:
text = 'Tokenizer Test is Started with Hugginface MarianTokenizer'
print(f"original : {text}")
print(f"token : {tokenizer.tokenize(text)}")

In [ ]:
text = '허깅페이스 마리안 토크나이저로 수행하는 토크나이저 테스트'
print(f"original : {text}")
print(f"token : {tokenizer.tokenize(text)}")

In [ ]:
text = '문장을 넣으면 토크나이즈해서 숫자로 바꿔줍니다.'

tokenized = tokenizer.tokenize(text)
encoded_tokens = tokenizer.encode(text, add_special_tokens=False)
encoded_tokens_end = tokenizer.encode(text, add_special_tokens=True)

print(tokenized)
print(encoded_tokens)
print(encoded_tokens_end)

In [ ]:
print(tokenizer.decode([0]))

In [ ]:
print(tokenizer.decode([13774]))

In [ ]:
tokenizer.decode(encoded_tokens)

# 3. Scheduler & Optimizer

## Noam Scheduler 공식

### $\text{Learning Rate} = \frac{1}{\sqrt{d_{\text{model}}}} \times \min\left(\frac{1}{\sqrt{\text{step\_num}}}, \frac{\text{step\_num}}{\text{warmup\_steps}^{1.5}}\right)$

In [ ]:
def count_params(model):
    num = sum([p.numel() for p in model.parameters() if p.requires_grad])
    return num

class NoamScheduler:
    def __init__(self, optimizer, d_model, warmup_steps, LR_scale=1):
        self.optimizer = optimizer  # 최적화할 옵티마이저
        self.step_count = 0  # 현재까지 진행된 스텝 수
        self.d_model = d_model  # 모델의 차원 수
        self.warmup_steps = warmup_steps  # 웜업 단계에서의 스텝 수
        self.LR_scale = LR_scale  # 학습률 스케일 인자
        self._d_model_factor = self.LR_scale * (self.d_model ** -0.5)  # 모델 차원에 대한 계수를 미리 계산

    def step(self):
        self.step_count += 1  # 스텝 수 증가
        lr = self.calculate_learning_rate()  # 새 학습률 계산
        self.optimizer.param_groups[0]['lr'] = lr  # 옵티마이저의 학습률 갱신

    def calculate_learning_rate(self):
        # 초기 웜업 단계에서는 학습률을 서서히 증가시키고, 이후에는 감소시키는 방식으로 계산
        minimum_factor = min(self.step_count ** -0.5, self.step_count * self.warmup_steps ** -1.5)
        return self._d_model_factor * minimum_factor
        
def plot_scheduler(scheduler_name, optimizer, scheduler, total_steps): # LR curve 보기
    lr_history = []
    steps = range(1, total_steps)

    for _ in steps: # base model -> 10만 steps (12시간), big model -> 30만 steps (3.5일) 로 훈련했다고 함
        lr_history += [optimizer.param_groups[0]['lr']]
        scheduler.step()

    plt.figure()
    if total_steps == 100000:
        plt.plot(steps, (512 ** -0.5) * torch.tensor(steps) ** -0.5, 'g--', linewidth=1, label=r"$d_{\mathrm{model}}^{-0.5} \cdot \mathrm{step}^{-0.5}$")
        plt.plot(steps, (512 ** -0.5) * torch.tensor(steps) * 4000 ** -1.5, 'r--', linewidth=1, label=r"$d_{\mathrm{model}}^{-0.5} \cdot \mathrm{step} \cdot \mathrm{warmup\_steps}^{-1.5}$")    
    plt.plot(steps, lr_history, 'b', linewidth=2, alpha=0.35, label="Learning Rate")

    plt.ylim([-0.1*max(lr_history), 1.2*max(lr_history)])
    plt.xlabel('Step')
    plt.ylabel('Learning Rate')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
optimizer = optim.Adam(nn.Linear(1, 1).parameters(), lr=0) # 테스트용 optimizer
scheduler = NoamScheduler(optimizer, d_model=512, warmup_steps=4000) # 논문 값
plot_scheduler(scheduler_name = 'Noam', optimizer = optimizer, scheduler = scheduler, total_steps = 100000)

optimizer = optim.Adam(nn.Linear(1, 1).parameters(), lr=0)
scheduler = NoamScheduler(optimizer, d_model=d_model, warmup_steps=warmup_steps, LR_scale=LR_scale)
plot_scheduler(scheduler_name = 'Noam', optimizer = optimizer, scheduler = scheduler, total_steps = int(len(train_DS)*EPOCH/BATCH_SIZE))

# 4. Regularization - Label Smoothing Loss

In [ ]:
def smooth_label(targets: torch.Tensor, classes: int, smoothing=0.1):
    assert 0 <= smoothing < 1
    confidence = 1.0 - smoothing
    label_shape = torch.Size((targets.size(0), classes))
    with torch.no_grad():
        smooth_labels = torch.empty(size=label_shape, device=targets.device)
        smooth_labels.fill_(smoothing / (classes - 1))
        smooth_labels.scatter_(1, targets.data.unsqueeze(1), confidence)
    return smooth_labels

def custom_cross_entropy(input, target, smoothing=0.1, ignore_index=-100):
    log_probs = F.log_softmax(input, dim=-1)
    target = smooth_label(target, input.size(-1), smoothing)
    
    if ignore_index >= 0:
        mask = target != ignore_index
        target = target[mask]
        log_probs = log_probs[mask]

    loss = (-target * log_probs).sum(dim=-1)
    return loss.mean()

In [ ]:
input = torch.randn(3, 5, requires_grad=True) # 임의의 예측값
target = torch.tensor([1, 0, 4])  # 실제 레이블
label = smooth_label(target, input.size(-1), 0.0)
label_smoothing = smooth_label(target, input.size(-1), 0.1)

loss = custom_cross_entropy(input, target, smoothing=0.0, ignore_index=pad_idx)
loss_smoothing = custom_cross_entropy(input, target, smoothing=0.1, ignore_index=pad_idx)
input, label, label_smoothing, loss, loss_smoothing

In [ ]:
class LabelSmoothingCrossEntropyLoss(nn.Module):
    def __init__(self, smoothing=0.1, ignore_index=65000):
        super(LabelSmoothingCrossEntropyLoss, self).__init__()
        # 스무딩 파라미터 설정. 0에 가까울수록 일반 크로스 엔트로피에 가까움
        self.smoothing = smoothing
        # 무시할 레이블(패딩)의 인덱스. 이 인덱스에 해당하는 레이블은 손실 계산에서 제외
        self.ignore_index = ignore_index

    def forward(self, input, target):
        # 입력 텍스트에 대한 로그 소프트맥스를 적용하여 모델의 예측 로그 확률을 계산
        log_probs = F.log_softmax(input, dim=-1)
        # 출력 언어의 어휘 크기를 계산 - 일반적인 분류 문제에서는 클래스의 수
        n_classes = input.size(-1)

        with torch.no_grad():
            # 스무딩된 레이블 분포를 생성. 각 클래스(어휘)에 작은 확률을 할당해 다양한 번역을 고려하도록 
            true_dist = torch.full_like(log_probs, self.smoothing / (n_classes - 1))
            # 무시할 레이블을 처리합니다. -> 패딩 토큰
            ignore = target == self.ignore_index
            # 무시할 레이블을 0으로 설정
            target = target.masked_fill(ignore, 0)
            # 실제 레이블 위치에 (1 - 스무딩) 값을 할당
            true_dist.scatter_(1, target.unsqueeze(1), 1.0 - self.smoothing)
            # 무시할 레이블의 위치에 0을 할당
            true_dist.masked_fill_(ignore.unsqueeze(1), 0)

            # 무시할 인덱스에 대한 마스크를 생성
            mask = ~ignore

        # 손실을 계산합니다. 마스크를 적용하여 무시할 인덱스를 제외
        loss = -true_dist * log_probs
        # 최종 손실을 평균내어 반환
        loss = loss.masked_select(mask.unsqueeze(1)).mean()

        return loss

In [ ]:
criterion = LabelSmoothingCrossEntropyLoss(smoothing=0.1, ignore_index=pad_idx)
# criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

# 5. Modeling

## 1) MHA

In [ ]:
class MHA(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()

        self.d_model = d_model
        self.n_heads = n_heads
        assert d_model % n_heads == 0, f'd_model ({d_model})은 n_heads ({n_heads})로 나누어 떨어져야 합니다.'

        self.head_dim = d_model // n_heads  # int 형변환 제거

        # 쿼리, 키, 값에 대한 선형 변환
        self.fc_q = nn.Linear(d_model, d_model) 
        self.fc_k = nn.Linear(d_model, d_model) 
        self.fc_v = nn.Linear(d_model, d_model)
        self.fc_o = nn.Linear(d_model, d_model)

        # 어텐션 점수를 위한 스케일 요소
        self.scale = torch.sqrt(torch.tensor(self.head_dim, dtype=torch.float32))

    def forward(self, Q, K, V, mask=None):
        batch_size = Q.shape[0]

        # 쿼리, 키, 값에 대한 선형 변환 수행
        Q = self.fc_q(Q) 
        K = self.fc_k(K)
        V = self.fc_v(V)

        # 멀티 헤드 어텐션을 위해 텐서 재구성 및 순서 변경
        Q = Q.reshape(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.reshape(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.reshape(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

        # 스케일드 닷-프로덕트 어텐션 계산
        attention_score = Q @ K.permute(0, 1, 3, 2) / self.scale

        # 마스크 적용 (제공된 경우)
        if mask is not None:
            attention_score = attention_score.masked_fill(mask, -1e10)

        # 소프트맥스를 사용하여 어텐션 확률 계산
        attention_dist = torch.softmax(attention_score, dim=-1)

        # 어텐션 결과
        attention = attention_dist @ V

        # 어텐션 헤드 재조립
        x = attention.permute(0, 2, 1, 3).reshape(batch_size, -1, self.d_model)

        # 최종 선형 변환
        x = self.fc_o(x)

        return x, attention_dist

## 2) Feed Forward Network

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, drop_p):
        super().__init__()

        self.linear = nn.Sequential(nn.Linear(d_model, d_ff),
                                    nn.ReLU(),
                                    nn.Dropout(drop_p),       ## ADD Dropout
                                    nn.Linear(d_ff, d_model))
    
    def forward(self, x):
        x = self.linear(x)
        return x

## 3) Encoder Components

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, drop_p):
        """
        EncoderLayer 클래스의 초기화 메소드입니다.
        :param d_model: 모델의 차원 크기
        :param n_heads: 어텐션 헤드의 개수
        :param d_ff: 피드 포워드 네트워크의 내부 차원
        :param drop_p: 드롭아웃 비율
        """
        super().__init__()

        self.self_atten = MHA(d_model, n_heads)
        self.FF = FeedForward(d_model, d_ff, drop_p)
        self.LN = nn.LayerNorm(d_model)

        self.dropout = nn.Dropout(drop_p)
    
    def forward(self, x, enc_mask):
        """
        EncoderLayer 클래스의 순전파 메소드입니다.
        :param x: 입력 텐서
        :param enc_mask: 인코더 마스크
        """
        x_norm = self.LN(x) ## Pre-LN
        
        # 멀티헤드 어텐션과 잔차 연결
        output, atten_enc = self.self_atten(x_norm, x_norm, x_norm, enc_mask)
        x = x + self.dropout(output)

        # 레이어 정규화 적용
        x_norm = self.LN(x)
        # 피드 포워드 네트워크와 잔차 연결
        output = self.FF(x_norm)
        x = x_norm + self.dropout(output)
        x = self.LN(x)

        return x, atten_enc

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_embedding, max_len, d_model, n_heads, n_layers, d_ff, drop_p):
        """
        Encoder 클래스의 초기화 메소드입니다.
        :param input_embedding: 입력 임베딩 레이어
        :param max_len: 입력 시퀀스의 최대 길이
        :param d_model: 모델의 차원 크기
        :param n_heads: 멀티헤드 어텐션의 헤드 수
        :param n_layers: 인코더 레이어의 수
        :param d_ff: 피드 포워드 네트워크의 내부 차원
        :param drop_p: 드롭아웃 비율
        """
        super().__init__()

        # 스케일링 팩터
        self.scale = torch.sqrt(torch.tensor(d_model, dtype=torch.float32))
        self.input_embedding = input_embedding
        self.pos_embedding = nn.Embedding(max_len, d_model)

        self.dropout = nn.Dropout(drop_p)

        # 인코더 레이어를 n_layers만큼 생성
        self.layers = nn.ModuleList([EncoderLayer(d_model, n_heads, d_ff, drop_p) for _ in range(n_layers)])        

    def forward(self, src, mask, atten_map_save=False):
        """
        Encoder 클래스의 순전파 메소드입니다.
        :param src: 입력 소스
        :param mask: 인코더 마스크
        :param atten_map_save: 어텐션 맵 저장 여부
        """
        pos = torch.arange(src.shape[1], device=src.device).repeat(src.shape[0], 1) # 위치 임베딩 생성

        x = self.scale * self.input_embedding(src) + self.pos_embedding(pos)
        x = self.dropout(x)
        
        atten_encs = []
        for layer in self.layers:
            x, atten_enc = layer(x, mask)
            if atten_map_save:
                atten_encs.append(atten_enc[0].unsqueeze(0))

        if atten_map_save:
            atten_encs = torch.cat(atten_encs, dim=0)

        return x, atten_encs


### Scale Effect of Input Embedding

In [ ]:
import torch
import matplotlib.pyplot as plt

# 임베딩 벡터와 d_model 설정
d_model = 512
embedding_vector = torch.randn(100, d_model)

# 스케일링 팩터 적용
scale = torch.sqrt(torch.tensor(d_model, dtype=torch.float32))
scaled_embedding_vector = scale * embedding_vector

# 임베딩 벡터의 분포를 히스토그램으로 시각화
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(embedding_vector.numpy().flatten(), bins=30, color='blue', alpha=0.7)
plt.title("Original Embedding Distribution")
plt.xlabel("Value")
plt.ylabel("Frequency")

plt.subplot(1, 2, 2)
plt.hist(scaled_embedding_vector.numpy().flatten(), bins=30, color='red', alpha=0.7)
plt.title("Scaled Embedding Distribution")
plt.xlabel("Value")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()


In [ ]:
def attention(query, key, scale=None):
    """
    간단한 어텐션 메커니즘을 구현한 함수입니다.
    :param query: 쿼리 벡터
    :param key: 키 벡터
    :param scale: 스케일링 팩터
    :return: 어텐션 스코어
    """
    d_k = query.size(-1)
    
    # query와 key의 내적 계산
    scores = torch.matmul(query, key.transpose(-2, -1))

    if scale:
        scores = scores / scale

    return torch.softmax(scores, dim=-1)

# 임의의 쿼리, 키, 밸류 생성
query = torch.randn(10, d_model)
key = torch.randn(10, d_model)

# 스케일링 적용 전후의 어텐션 스코어 계산
attention_scores_without_scaling = attention(query, key)
attention_scores_with_scaling = attention(query, key, scale)

# 어텐션 스코어의 분포를 히스토그램으로 시각화
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(attention_scores_without_scaling.numpy().flatten(), bins=30, color='blue', alpha=0.7)
plt.title("Attention Scores Without Scaling")
plt.xlabel("Score")
plt.ylabel("Frequency")

plt.subplot(1, 2, 2)
plt.hist(attention_scores_with_scaling.numpy().flatten(), bins=30, color='red', alpha=0.7)
plt.title("Attention Scores With Scaling")
plt.xlabel("Score")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()


## 4) Decoder Components

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, drop_p):
        """
        DecoderLayer 클래스의 초기화 메소드입니다.
        :param d_model: 모델의 차원 크기
        :param n_heads: 멀티헤드 어텐션의 헤드 수
        :param d_ff: 피드 포워드 네트워크의 내부 차원
        :param drop_p: 드롭아웃 비율
        """
        super().__init__()        
        self.atten = MHA(d_model, n_heads) # Attention for Self & Cross
        self.FF = FeedForward(d_model, d_ff, drop_p) # ff network
        self.LN = nn.LayerNorm(d_model) # Layer Normalization
        self.dropout = nn.Dropout(drop_p) # Dropout

    def forward(self, x, enc_out, dec_mask, enc_dec_mask):
        """
        DecoderLayer 클래스의 순전파 메소드입니다.
        :param x: 디코더의 입력
        :param enc_out: 인코더의 출력
        :param dec_mask: 디코더 마스크
        :param enc_dec_mask: 인코더-디코더 마스크
        """
        x, atten_dec = self.process_sublayer(x, self.atten, self.LN, dec_mask)
        x, atten_enc_dec = self.process_sublayer(x, self.atten, self.LN, enc_dec_mask, enc_out)
        x, _ = self.process_sublayer(x, self.FF, self.LN)

        return x, atten_dec, atten_enc_dec

    def process_sublayer(self, x, sublayer, norm_layer, mask=None, enc_out=None):
        """
        디코더의 서브레이어 처리를 위한 함수.
        :param x: 입력 텐서
        :param sublayer: 서브레이어 (어텐션 또는 피드 포워드)
        :param norm_layer: 레이어 정규화
        :param mask: 마스크 (디코더 또는 인코더-디코더 마스크)
        :param enc_out: 인코더의 출력 (인코더-디코더 어텐션에만 필요)
        """
        x_norm = norm_layer(x)
        if isinstance(sublayer, MHA): # mha case
            if enc_out is not None: # encoder-decoder attention
                residual, atten = sublayer(x_norm, enc_out, enc_out, mask)
            else: # self attention
                residual, atten = sublayer(x_norm, x_norm, x_norm, mask)
        elif isinstance(sublayer, FeedForward): # ff network
            residual = sublayer(x_norm)
            atten = None  # 피드 포워드 레이어는 어텐션 맵을 반환하지 않음
        else:
            raise TypeError("Unsupported sublayer type")

        return x + self.dropout(residual), atten

In [ ]:
class Decoder(nn.Module):
    def __init__(self, input_embedding, max_len, d_model, n_heads, n_layers, d_ff, drop_p):
        """
        Decoder 클래스의 초기화 메소드.
        :param input_embedding: 입력 임베딩 레이어
        :param max_len: 입력 시퀀스의 최대 길이
        :param d_model: 모델의 차원 크기
        :param n_heads: 멀티헤드 어텐션의 헤드 수
        :param n_layers: 디코더 레이어의 수
        :param d_ff: 피드 포워드 네트워크의 내부 차원
        :param drop_p: 드롭아웃 비율
        """
        super().__init__()

        self.scale = torch.sqrt(torch.tensor(d_model, dtype=torch.float32))
        self.input_embedding = input_embedding
        self.pos_embedding = nn.Embedding(max_len, d_model)
        self.dropout = nn.Dropout(drop_p)

        self.layers = nn.ModuleList([DecoderLayer(d_model, n_heads, d_ff, drop_p) for _ in range(n_layers)])

        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, trg, enc_out, dec_mask, enc_dec_mask, atten_map_save=False):
        """
        Decoder 클래스의 순전파 메소드.
        :param trg: 타깃 입력
        :param enc_out: 인코더의 출력
        :param dec_mask: 디코더 마스크
        :param enc_dec_mask: 인코더-디코더 마스크
        :param atten_map_save: 어텐션 맵 저장 여부
        """
        pos = torch.arange(trg.shape[1], device=trg.device).repeat(trg.shape[0], 1)

        x = self.scale * self.input_embedding(trg) + self.pos_embedding(pos)
        x = self.dropout(x)

        atten_decs = []
        atten_enc_decs = []
        for layer in self.layers:
            x, atten_dec, atten_enc_dec = layer(x, enc_out, dec_mask, enc_dec_mask)
            if atten_map_save:
                atten_decs.append(atten_dec[0].unsqueeze(0))
                atten_enc_decs.append(atten_enc_dec[0].unsqueeze(0))

        if atten_map_save:
            atten_decs = torch.cat(atten_decs, dim=0)
            atten_enc_decs = torch.cat(atten_enc_decs, dim=0)

        x = self.fc_out(x)
        
        return x, atten_decs, atten_enc_decs


## 5) Transformer

In [ ]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, max_len, d_model, n_heads, n_layers, d_ff, drop_p):
        """
        Transformer 클래스의 초기화 메소드.
        :param vocab_size: 어휘 사전의 크기
        :param max_len: 입력 시퀀스의 최대 길이
        :param d_model: 모델의 차원 크기
        :param n_heads: 멀티헤드 어텐션의 헤드 수
        :param n_layers: 인코더 및 디코더 레이어의 수
        :param d_ff: 피드 포워드 네트워크의 내부 차원
        :param drop_p: 드롭아웃 비율
        """
        super().__init__()

        input_embedding = nn.Embedding(vocab_size, d_model) 
        self.encoder = Encoder(input_embedding, max_len, d_model, n_heads, n_layers, d_ff, drop_p)
        self.decoder = Decoder(input_embedding, max_len, d_model, n_heads, n_layers, d_ff, drop_p)

        self.n_heads = n_heads

        # 파라미터 초기화
        for m in self.modules():
            if hasattr(m, 'weight') and m.weight.dim() > 1: 
                nn.init.xavier_uniform_(m.weight) 

    def make_enc_mask(self, src):
        """
        인코더 마스크 생성.
        :param src: 입력 소스 (batch_size, src_len)
        :return: 인코더 마스크 (batch_size, 1, 1, src_len)
                 - pad_idx에 해당하는 위치는 True, 그 외는 False
        """
        enc_mask = (src == pad_idx).unsqueeze(1).unsqueeze(2)
        return enc_mask.repeat(1, self.n_heads, src.shape[1], 1).to(src.device)

    def make_dec_mask(self, trg):
        """
        디코더 마스크 생성 (패딩 마스크 및 미래 토큰 마스킹).
        :param trg: 타깃 입력 (batch_size, trg_len)
        :return: 디코더 마스크 (batch_size, 1, trg_len, trg_len)
                 - 패딩 위치 및 미래 위치는 True, 그 외는 False
        """
        trg_pad_mask = (trg == pad_idx).unsqueeze(1).unsqueeze(2)
        trg_pad_mask = trg_pad_mask.repeat(1, self.n_heads, trg.shape[1], 1).to(trg.device)
        trg_dec_mask = torch.tril(torch.ones(trg.shape[0], self.n_heads, trg.shape[1], trg.shape[1], device=trg.device))==0
        dec_mask = trg_pad_mask | trg_dec_mask
        return dec_mask

    def make_enc_dec_mask(self, src, trg):
        """
        인코더-디코더 마스크 생성.
        :param src: 입력 소스 (batch_size, src_len)
        :param trg: 타깃 입력 (batch_size, trg_len)
        :return: 인코더-디코더 마스크 (batch_size, 1, trg_len, src_len)
                 - 소스의 pad_idx 위치는 True, 그 외는 False
        """
        enc_dec_mask = (src == pad_idx).unsqueeze(1).unsqueeze(2)
        return enc_dec_mask.repeat(1, self.n_heads, trg.shape[1], 1).to(src.device)

    def forward(self, src, trg):
        enc_mask = self.make_enc_mask(src)
        dec_mask = self.make_dec_mask(trg)
        enc_dec_mask = self.make_enc_dec_mask(src, trg)

        enc_out, atten_encs = self.encoder(src, enc_mask)
        out, atten_decs, atten_enc_decs = self.decoder(trg, enc_out, dec_mask, enc_dec_mask)

        return out, atten_encs, atten_decs, atten_enc_decs


In [ ]:
save_model_path = './translator_ls.pt'
save_history_path = './translator_history_ls.pt'

In [ ]:
DEVICE = 'cuda:0' ## 8대의 GPU 없음

# BATCH_SIZE = 128 ## 논문에선 2.5만 token이 한 batch에 담기게 했다고 함.
# EPOCH = 20 ## 논문에선 약 560 에포크 진행
# max_len = 512 

# warmup_steps = 1500 ## 논문에선 4,000 스탭 
# LR_scale = 1 # Noam scheduler에 peak LR 값 조절을 위해 곱해질 Scale 

In [ ]:
# 논문에 나오는 base 모델
d_model = 512
n_heads = 8
n_layers = 6
d_ff = 2048
drop_p = 0.1

# 좀 사이즈 줄인 모델 
# base model params와 맞춤 : 65 mil
d_model = 400
n_heads = 8
n_layers = 4
d_ff = 1200
drop_p = 0.1

# 사이즈 더 줄인 모델 
d_model = 256
n_heads = 8
n_layers = 4
d_ff = 512
drop_p = 0.1

In [ ]:
# test_DS 테스트
i = 5
idx = test_DS.indices[i]
print(idx) # 엑셀 파일에서 idx번째 문장에 들어있음을 확인할 수 있다
src_text, trg_text = custom_DS.__getitem__(idx)
print(src_text)
print(trg_text)

In [ ]:
def Train(model, train_DL, val_DL, criterion, optimizer):
    history = {"train": [], "val": [], "lr":[]}
    best_loss = float('inf')

    for ep in range(EPOCH):
        start_time = time.time()  # 에포크 시작 시간 기록

        # 학습 모드
        model.train()
        train_loss = loss_epoch(model, train_DL, criterion, optimizer=optimizer, max_len=max_len, DEVICE=DEVICE, tokenizer=tokenizer)
        history["train"].append(train_loss)

        # 현재 학습률 기록
        current_lr = optimizer.param_groups[0]['lr']
        history["lr"].append(current_lr)
        
        # 평가 모드
        model.eval()
        with torch.no_grad():
            val_loss = loss_epoch(model, val_DL, criterion, max_len=max_len, DEVICE=DEVICE, tokenizer=tokenizer)
            history["val"].append(val_loss)
            epoch_time = time.time() - start_time

            # 로그 출력
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save({"model": model, "ep": ep, "optimizer": optimizer.state_dict(), 'loss':val_loss}, save_model_path)
                print(f"| Epoch {ep+1}/{EPOCH} | train loss:{train_loss:.5f} val loss:{val_loss:.5f} current_LR:{optimizer.param_groups[0]['lr']:.8f} time:{epoch_time:.2f}s => Model Saved!")
            else :
                print(f"| Epoch {ep+1}/{EPOCH} | train loss:{train_loss:.5f} val loss:{val_loss:.5f} current_LR:{optimizer.param_groups[0]['lr']:.8f} time:{epoch_time:.2f}s")

    torch.save({"loss_history": history, "EPOCH": EPOCH, "BATCH_SIZE": BATCH_SIZE}, save_history_path)
    
    show_history(loss_history=history)
    
def show_history(history, save_path='train_history_ls'):
    # train loss, val loss 시각화
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(1, EPOCH + 1)), y=history["train"], mode='lines+markers', name='Train Loss'))
    fig.add_trace(go.Scatter(x=list(range(1, EPOCH + 1)), y=history["val"], mode='lines+markers', name='Validation Loss'))

    fig.update_layout(
        title='Training History',
        xaxis_title='Epoch',
        yaxis=dict(title='Loss'),
        showlegend=True
    )
    fig.write_image(save_path+".png")
    fig.show()
    
    # learning rate 시각화
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(1, EPOCH + 1)), y=history['lr'], mode='lines+markers', name='Learning Rate'))

    # 레이아웃 업데이트
    fig.update_layout(
        title='Training History',
        xaxis_title='Epoch',
        yaxis=dict(title='Learning Rate'),
        showlegend=True
    )
    fig.write_image(save_path+"_lr.png")
    fig.show()
    

def loss_epoch(model, DL, criterion, optimizer=None, max_len=None, DEVICE=None, tokenizer=None):
    N = len(DL.dataset) # 데이터 수

    rloss = 0
    for src_texts, trg_texts in tqdm(DL, leave=False):
        src = tokenizer(src_texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt').input_ids.to(DEVICE)
        trg_texts = ['</s> ' + s for s in trg_texts]
        trg = tokenizer(trg_texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt').input_ids.to(DEVICE)
        
        # inference
        y_hat = model(src, trg[:, :-1])[0] # 모델 통과 시킬 때 trg의 <eos>는 제외!
        loss = criterion(y_hat.permute(0, 2, 1), trg[:, 1:]) # 손실 계산 시 <sos> 는 제외!
        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
        
        # loss accumulation
        loss_b = loss.item() * src.shape[0]
        rloss += loss_b
    loss_e = rloss / N
    return loss_e

def Test(model, test_DL, criterion, max_len, DEVICE, tokenizer):
    model.eval() # test mode로 전환
    with torch.no_grad():
        test_loss = loss_epoch(model, test_DL, criterion, max_len=max_len, DEVICE=DEVICE, tokenizer=tokenizer)
    print(f"Test loss: {round(test_loss, 3)} | Test PPL: {round(math.exp(test_loss), 3)}")

In [ ]:
model = Transformer(vocab_size, max_len, d_model, n_heads, n_layers, d_ff, drop_p).to(DEVICE)

# # 모델의 레이어와 파라미터 출력
# for name, module in model.named_modules():
#     print(name, module)

In [ ]:
# 총 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())

print(f"Total parameters: {total_params:,}")

In [ ]:
params = model.parameters()

# 논문에서 제시한 beta와 eps 사용 & 맨 처음 step 의 LR=0으로 출발 (warm-up)
optimizer = optim.Adam(params, 
                       lr=0, 
                       betas=(0.9, 0.98), 
                       eps=1e-9) 
scheduler = NoamScheduler(optimizer, d_model=d_model, warmup_steps=warmup_steps, LR_scale=LR_scale)


Train(model, train_DL, val_DL, criterion, optimizer)

In [ ]:
loaded = torch.load('translator.pt', map_location=DEVICE)
load_model = loaded['model']
ep = loaded['ep']
optimizer = loaded['optimizer']

print(loaded.keys())

In [ ]:
def translation(model, src_text, atten_map_save=False, extra_token_length=50):
    model.eval()
    with torch.no_grad():
        src = tokenizer.encode(src_text, return_tensors='pt').to(DEVICE) 
        enc_mask = model.make_enc_mask(src)
        enc_out, atten_enc = model.encoder(src, enc_mask, atten_map_save)

        # 입력 시퀀스의 길이 계산 및 출력 시퀀스의 최대 길이 설정
        max_output_length = src.shape[1] + extra_token_length

        pred = tokenizer.encode('</s>', return_tensors='pt', add_special_tokens=False).to(DEVICE)
        for _ in range(max_output_length):
            dec_mask = model.make_dec_mask(pred)
            enc_dec_mask = model.make_enc_dec_mask(src, pred)
            out, atten_dec, atten_enc_dec = model.decoder(pred, enc_out, dec_mask, enc_dec_mask, atten_map_save)

            pred_word = out.argmax(dim=2)[:,-1].unsqueeze(0) 
            pred = torch.cat([pred, pred_word], dim=1) 

            if tokenizer.decode(pred_word.item()) == '</s>':
                break

        translated_text = tokenizer.decode(pred[0])

    return translated_text, atten_enc, atten_dec, atten_enc_dec


def show_attention(atten, Query, Key, n):
    plt.rc('font', family='NanumBarunGothic')
    atten = atten.cpu()

    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=[atten.shape[3]*1.5,atten.shape[2]])
    for i in range(3):
        ax[i].set_yticks(range(atten.shape[2]))
        ax[i].set_yticklabels(Query, rotation=45)
        ax[i].set_xticks(range(atten.shape[3]))
        ax[i].set_xticklabels(Key, rotation=60)
        ax[i].imshow(atten[n][i], cmap='bone') # h 번째 layer, 앞 세 개의 헤드만 plot
        # ax[i].xaxis.tick_top()  # x축 레이블을 위쪽으로 이동

In [ ]:
# 번역해보기
i = 1
idx = test_DS.indices[i]
src_text, trg_text = custom_DS.__getitem__(idx)
print(f"입력: {src_text}")
print(f"정답: {trg_text}")

translated_text, atten_enc, atten_dec, atten_enc_dec = translation(load_model, src_text, atten_map_save = True)
print(f"AI의 번역: {translated_text}")

In [ ]:
enc_input = tokenizer.tokenize(src_text+' </s>') # <eos> 붙여서 학습 시켰기 때문에 여기도 붙여줘야
dec_tokens = tokenizer.tokenize(translated_text) 
dec_input = dec_tokens[:-1] # 디코더 입력으로 들어가는 문장(sos 는 있고 eos는 없고)
dec_output = dec_tokens[1:] # 디코더 출력으로 나간 문장

show_attention(atten_enc, enc_input, enc_input, n = -1)
show_attention(atten_dec, dec_input, dec_input, n = -1)
show_attention(atten_enc_dec, dec_output, enc_input, n = -1) # 이 map을 해석할 때는 "이 단어가 나오게끔 뭘 주목했느냐" 로 해석해줘야 함 (ytick에 들어가는 단어가 아닌 예측한 단어를 썼기 때문)

In [ ]:
from torchtext.data.metrics import bleu_score

# trgs = [[['훌륭한', '강사와', '훌륭한', '수강생이','만나면','명강의가', '탄생한다']]]
# preds = [['훌륭한', '강사와', '훌륭한', '수강생이','함께라면','명강의가','만들어진다']]
# preds = [['만들어진다', '강사와', '훌륭한', '명강의가','훌륭한','수강생이','함께라면']]
# preds = [['훌륭한', '강사와', '훌륭한', '수강생이','훌륭한','강사와','훌륭한','강의를', '만든다']]
# preds = [['수강생이', '만나면', '명강의가', '탄생한다']]

trgs = [[['훌륭한', '강사와', '훌륭한', '수강생이','만나면','명강의가', '탄생한다']], [['이것은', '두','번째','문장입니다']]]
preds = [['훌륭한', '강사와', '훌륭한', '수강생이','훌륭한','강의를','만든다'], ['이것은','문장입니다']]

bleu_score(preds, trgs, max_n = 4, weights = [0.25,0.25,0.25,0.25]) # default
# bleu_score(preds, trgs, max_n = 1, weights = [1])

In [ ]:
def calc_bleu_score(model, DS):
    trgs = []
    preds = []

    for i, (src_text, trg_text) in enumerate(DS):
        
        translated_text, _, _, _ = translation(load_model, src_text)

        trg = tokenizer.tokenize(trg_text)
        translated_tok = tokenizer.tokenize(translated_text)[1:-1] # <sos> & <eos> 제외

        trgs += [[trg]]
        preds += [translated_tok] 
        
        if (i + 1) % 100 == 0:
            print(f"[{i + 1}/{len(DS)}]")
            print(f"입력: {src_text}")
            print(f"정답: {trg_text}")
            print(f"AI의 번역: {translated_text[5:-4]}") # 문자열에서 </s> 안보이게 하려고..

    bleu = bleu_score(preds, trgs)
    print()
    print(f'Total BLEU Score = {bleu*100:.2f}')

In [ ]:
calc_bleu_score(load_model, test_DS)

In [ ]:
# 내 번역기 써보기!
src_text = "안녕하세요! 이 강의 정말 열심히 준비 했어요."
print(f"입력: {src_text}")

translated_text, _, _, _ = translation(load_model, src_text)
print(f"AI의 번역: {translated_text}")